## SRJ 

## Hadamard test |HF> + G5|HF>

In [ ]:
import pennylane as qml
from pennylane import numpy as np

n_wires = 5         # 3 data wires + 1 ancilla
ancilla = 4
data_wires = [0,1,2,3]
hf_occ = np.array([1, 1, 0,0]) # 2 electrons in 3 orbitals

dev = qml.device('lightning.qubit', wires=n_wires)

@qml.qnode(dev)
def superposition_state():
    # Step 1: Prepare ancilla in superposition (Hadamard)
    qml.Hadamard(wires=ancilla)

    # Step 2: Controlled preparation
    # If ancilla = 0, do nothing (wires default to |000> = HF state)
    # If ancilla = 1, apply G5 (e.g., double excitation) to data

    # Prepare |HF> (basis state) on data wires
    qml.BasisState(hf_occ, wires=data_wires)

    # Apply G5 excitation, *controlled on ancilla=1*
    qml.ctrl(qml.DoubleExcitation, control=ancilla)(np.pi/2, wires=data_wires)  # Example excitation

    return qml.state()

s = superposition_state()

print("Full joint statevector (ancilla ⊗ data wires):", s)
# To get the data wires only in the superposed state, ideally you'd project onto ancilla in |+⟩

dev1 = qml.device('lightning.qubit', wires=6, shots = 1000)
@qml.qnode(dev1)
def measure(s):
    qml.StatePrep(s, wires=data_wires)  # Prepare the state on data wires
    return qml.counts()

counts = measure(s)
print(counts)


In [1]:
# Preparing the term1:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        #print('HF stat:', hf_state)
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('The ash excitation now is', ash_excitation[i])
                #print('Wires1 Invert removed are ', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)) )
                #print('Wires2 Invert removed are',list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    #print('Null state is', null_state)
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=1) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668933
Total excitations are 26
The adapt iteration now is 0
Highest gradient excitation is [2, 3, 6, 7]
Final updated parameters: [1.12299267]
Final cost: -1.4300836708403657
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(1.12)─┤  State
3: ──X─├FermionicDoubleExcitation(1.12)─┤  State
6: ────├FermionicDoubleExcitation(1.12)─┤  State
7: ────╰FermionicDoubleExcitation(1.12)─┤  State
The params after GS is [1.12299267]
Ash excitation after gs state: [[2, 3, 6, 7]]
Energies: [-1.4300836708403657]
Old gradient: [0.3100451588582031]


## State1 preparation

In [2]:
ashs = [[2,3,6,7], [0,3,5,6], [0,1,4,5]]
print('Exc. going to prepare state1',ashs) 
hf_state = np.array([1,1,1,1,0,0,0,0])
qubits = 8
params = np.zeros(len(ashs), requires_grad=True)
dev = qml.device("lightning.qubit", wires=qubits)

@qml.qnode(dev)
def s1(params, ashs, hf_state, H):
    #print('HF stat:', hf_state)
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
    for i, excitation in enumerate(ashs):
        #print('The value of i', i)
        if len(ashs[i]) == 4:
            #print('The ash excitation now is', ashs[i])
            qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
        elif len(ashs[i]) == 2:
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
    return qml.expval(H)  #Calculating the expectation value of the Hamiltonian

def cost(params):
    energy = s1(params, ashs, hf_state, H)
    return energy

result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

params = result.x
print("Final updated parameters for new excitations:", params)
print("Expectation value with new excitations:", result.fun)

term1 = result.fun
print('Term1 energy:', term1)

#State preparation after adding states and params
dev1 = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev1)
def s1state(hf_state, ashs, params):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
    for i, excitations in enumerate(ashs):
        if len(ashs[i]) == 4:
            qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
        elif len(ashs[i]) == 2:
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
    return qml.state()
s1ostate = s1state(hf_state, ashs, params)
print(qml.draw(s1state, max_length=100)(hf_state,ashs,params))

Exc. going to prepare state1 [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5]]
Final updated parameters for new excitations: [1.20339418 1.38269691 1.45819168]
Expectation value with new excitations: -1.7436334426150542
Term1 energy: -1.7436334426150542
0: ──X──────────────────────────────────╭FermionicDoubleExcitation(1.38)
1: ──X──────────────────────────────────├FermionicDoubleExcitation(1.38)
2: ──X─╭FermionicDoubleExcitation(1.20)─├FermionicDoubleExcitation(1.38)
3: ──X─├FermionicDoubleExcitation(1.20)─├FermionicDoubleExcitation(1.38)
4: ────│────────────────────────────────│───────────────────────────────
5: ────│────────────────────────────────├FermionicDoubleExcitation(1.38)
6: ────├FermionicDoubleExcitation(1.20)─╰FermionicDoubleExcitation(1.38)
7: ────╰FermionicDoubleExcitation(1.20)─────────────────────────────────

──╭FermionicDoubleExcitation(1.46)─┤  State
──├FermionicDoubleExcitation(1.46)─┤  State
──│────────────────────────────────┤  State
──│────────────────────────────────┤

## State 2 preparation

In [3]:
latest_ash = [ashs[-1]]
print("Latest excitation:", latest_ash)
print('Length of latest excitation:', len(latest_ash))
params2 = np.zeros(len(latest_ash), requires_grad=True)
print(params2)
#State 2
@qml.qnode(dev)
def s2(hf_state, latest_ash, params2, H):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    if len(latest_ash) == 4:
        #print('The latest ash excitation is', ashs[-1])
        qml.DoubleExcitation(params2, wires = ashs[-1])  # Assuming latest_ash is a list of lists, each containing the wires for the excitation
    elif len(latest_ash) == 2:
        qml.SingleExcitation(params2, wires = ashs[-1]) 
    return qml.expval(H)

def cost2(params2):
    energy = s2(hf_state, ashs[-1], params2, H)
    return energy
result2= minimize(cost2, params2, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

print("Final updated parameters for latest excitation:", result2.x)
print("Expectation value with new excitations:", result2.fun)

params2 = result2.x
print('Params2 is', params2)
term2 = result2.fun
print('Term2 is', term2)
dev1 = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev1)
def s2state(hf_state, ashs, params2):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
    if len(latest_ash[0]) == 4:
        print('The latest ash excitation is', latest_ash[0])
        print('params2 is', params2)
        qml.DoubleExcitation(params2, wires = latest_ash[0]) 
    elif len(latest_ash[0]) == 2:
        qml.SingleExcitation(params2, wires = latest_ash[0])
    return qml.state()

s2ostate = s2state(hf_state, latest_ash[0], params2)
print(qml.draw(s2state, max_length=100)(hf_state,latest_ash[0],params2))

Latest excitation: [[0, 1, 4, 5]]
Length of latest excitation: 1
[0.]
Final updated parameters for latest excitation: [1.12229166]
Expectation value with new excitations: -1.4286572071381494
Params2 is [1.12229166]
Term2 is -1.4286572071381494
The latest ash excitation is [0, 1, 4, 5]
params2 is [1.12229166]
The latest ash excitation is [0, 1, 4, 5]
params2 is [1.12229166]
0: ──X─╭G²(M0)─┤  State
1: ──X─├G²(M0)─┤  State
2: ──X─│───────┤  State
3: ──X─│───────┤  State
4: ────├G²(M0)─┤  State
5: ────╰G²(M0)─┤  State

M0 = 
[1.12229166]


In [4]:
s1ostatec = s1ostate.conj()
print('s1ostate is', s1ostatec.shape)  


s1ostate is (256,)


In [ ]:
@qml.qnode(dev)
def checking():
    qml.BasisState(hf_state, wires=range(qubits))
    qml.DoubleExcitation(result2.x, wires=[0,1,4,5])

    return qml.expval(H)

print("Expectation value for checking:", checking())

In [5]:
# Remove unwanted .T or extra shape operations
# If s1ostate or s2ostate are (1,256), flatten:
if s1ostatec.shape == (1, 256):
    s1_vec = s1ostatec.flatten()
else:
    s1_vec = s1ostatec
if s2ostate.shape == (1, 256):
    s2_vec = s2ostate.flatten()
else:
    s2_vec = s2ostate


In [9]:
#Generating term 3
Ham_matrix = qml.matrix(H, wire_order=range(qubits))
print("Hamiltonian matrix:", Ham_matrix.shape)
left_op = s1ostate.T.conj()
print('Left operator shape:', left_op.shape)
right_op = s2ostate
print('Right operator shape:', right_op.shape)

# Calculate the expectation value of the Hamiltonian with respect to the combined state
#term3_exp = np.vdot(left_opright_op).real
#term3_exp = np.dot(left_op, np.dot(Ham_matrix, right_op)).real
#term3_exp = left_op.dot(Ham_matrix.dot(right_op)).real 
term3_exp = np.vdot(s1_vec, Ham_matrix @ s2_vec).real
print("Expectation value of term 3:", term3_exp)



# Final energy calculation
H_sum = term1 + term2 + (2*term3_exp)
print(H_sum)

Hamiltonian matrix: (256, 256)
Left operator shape: (256,)
Right operator shape: (1, 256)
Expectation value of term 3: -1.0938621849121033
-5.3600150195774106


## Calculating normalization factor

In [16]:
D1 = np.vdot(s1ostatec, s1ostate).real
print('Normalization of s1 state:', D1)

D2 = np.vdot(s2ostate.conj(), s2ostate).real
print('Normalization of s2 state:', D2)

D3 = np.vdot(s1ostatec, s2ostate).real
print('Overlap between s1 and s2:', D3)

D_sum_root = (D1 + D2 + (2*D3))**0.5
D_sum = D1 + D2 + (2*D3)
print('D_sum:', D_sum)
print('D_sum_root:', D_sum_root)

Normalization of s1 state: 0.9999999999999825
Normalization of s2 state: 1.0
Overlap between s1 and s2: 0.6261526642786085
D_sum: 3.2523053285571994
D_sum_root: 1.8034149074900094


In [17]:
# Final energy with normalization
final_energy = H_sum / (D_sum)
print("Final energy with normalization:", final_energy)

Final energy with normalization: -1.6480663646531741
